In [128]:
import json

In [162]:
def tweettotable(input_tweets,input_table): #program does nothing as written
    x=[]
    with open(input_tweets) as f1:
        for line in f1:
            x.append(json.loads(line))
    f2 = open(input_table, 'w')
    for t in range(len(x)):
        f2.write('\t'.join([x[t]['created_at'].encode("utf8"), x[t]['text'].encode("utf8").replace('\n',''),'\n']))    

In [163]:
tweettotable('data/tweets.json','data/input_table.csv')